In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from scipy.optimize import least_squares
from scipy.optimize import minimize_scalar

In [2]:
option_chains_train = pd.read_csv("/kaggle/input/raw-option-chains/aapl_2016_2020.csv")
option_chains_test = pd.read_csv("/kaggle/input/raw-option-chains/aapl_2021_2023.csv")
risk_free_rates = pd.read_csv("/kaggle/input/fed-funds-rates/FedFunds History.csv", sep = ";")

/tmp/ipykernel_47/4097170841.py:1: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  option_chains_train = pd.read_csv("/kaggle/input/raw-option-chains/aapl_2016_2020.csv")
/tmp/ipykernel_47/4097170841.py:2: DtypeWarning: Columns (15,17,18,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  option_chains_test = pd.read_csv("/kaggle/input/raw-option-chains/aapl_2021_2023.csv")


In [3]:
def clean_data (df):
  cols = [" [QUOTE_DATE]"," [EXPIRE_DATE]"," [DTE]"," [C_IV]"," [C_BID]"," [C_ASK]"," [STRIKE]"," [P_BID]"," [P_ASK]"," [P_IV]"," [UNDERLYING_LAST]"]
  df = df[cols]
  df.columns = ["QUOTE_DATE", "EXPIRE_DATE", "DTE","C_IV","C_BID","C_ASK","STRIKE","P_BID","P_ASK","P_IV","UNDERLYING_LAST"]
  df = df.dropna()
  df["QUOTE_DATE"] = pd.to_datetime(df["QUOTE_DATE"])
  df["EXPIRE_DATE"] = pd.to_datetime(df["EXPIRE_DATE"])
  df["TTM"] = df["DTE"].round()/365
  df["RELATIVE_STRIKE"] = df["STRIKE"] / df["UNDERLYING_LAST"]
  df["C_IV"] = df["C_IV"].apply(pd.to_numeric, errors="coerce").astype(float)
  df["C_BID"] = df["C_BID"].apply(pd.to_numeric, errors="coerce").astype(float)
  df["C_ASK"] = df["C_ASK"].apply(pd.to_numeric, errors="coerce").astype(float)
  df["P_IV"] = df["P_IV"].apply(pd.to_numeric, errors="coerce").astype(float)
  df["P_BID"] = df["P_BID"].apply(pd.to_numeric, errors="coerce").astype(float)
  df["P_ASK"] = df["P_ASK"].apply(pd.to_numeric, errors="coerce").astype(float)
  df = df.dropna()
  return df

In [4]:
def rates_clean(df):
  df["Date"] = pd.to_datetime(df["Date"])
  df["Rate"] = df["Rate"].str.replace(",",".").astype(float)/100
  df = df.sort_values("Date").set_index("Date")
  full_idx = pd.date_range(df.index.min(), df.index.max(), freq="D")
  df_mod = df.reindex(full_idx)
  num_cols = df_mod.select_dtypes("number").columns
  df_mod[num_cols] = df_mod[num_cols].interpolate(method="time").ffill().bfill()
  df_mod = df_mod.rename_axis("Date").reset_index()
  return df_mod

In [5]:
def df_option_chains_rates_cleaning(df):
    df["F"] = df["UNDERLYING_LAST"] * np.exp((df["Rate"]) * df["TTM"])
    df["LogMK"] = np.log(df["STRIKE"]/df["F"])
    df["w"] = (df["C_IV"]**2) * df["TTM"]
    df.drop(columns=["Date"], inplace=True)
    return df

In [6]:
def dates_extraction(df):
  dates = df["QUOTE_DATE"].drop_duplicates().sort_values(ascending=True)
  return dates

In [7]:
def get_price(df,date):
  price = df.loc[df["QUOTE_DATE"].eq(date), "UNDERLYING_LAST"].iloc[0]
  return price

In [8]:
def get_rate(df,date):
  rate = df.loc[df["Date"].eq(date), "Rate"].iloc[0]
  return rate

In [9]:
def essvi_total_variance(k, theta, psi, rho):
    k = np.asarray(k, dtype=float)
    if theta <= 0.0 or psi <= 0.0 or abs(rho) >= 1.0:
        return np.full_like(k, np.nan, dtype=float)
    phi = psi / theta
    x = phi * k + rho
    inside = x * x + (1.0 - rho * rho)
    inside = np.maximum(inside, 1e-16)
    w = 0.5 * theta * (1.0 + rho * phi * k + np.sqrt(inside))
    return w

def interpolate_essvi_params(T_target, Ts, theta, psi, rho):
    Ts = np.asarray(Ts, dtype=float)
    theta = np.asarray(theta, dtype=float)
    psi = np.asarray(psi, dtype=float)
    rho = np.asarray(rho, dtype=float)
    T_target = float(T_target)

    if T_target <= Ts[0]:
        lam = T_target / Ts[0] if Ts[0] > 0 else 0.0
        theta_T = lam * theta[0]
        psi_T = lam * psi[0]
        rho_T = rho[0]
        return theta_T, psi_T, rho_T

    if T_target >= Ts[-1]:
        if len(Ts) >= 2 and Ts[-1] > Ts[-2]:
            slope = max((theta[-1] - theta[-2]) / (Ts[-1] - Ts[-2]), 0.0)
        else:
            slope = 0.0
        theta_T = theta[-1] + slope * (T_target - Ts[-1])
        psi_T = psi[-1]
        rho_T = rho[-1]
        return theta_T, psi_T, rho_T

    i = np.searchsorted(Ts, T_target) - 1
    i = max(0, min(i, len(Ts) - 2))
    T0, T1 = Ts[i], Ts[i + 1]
    lam = (T_target - T0) / (T1 - T0)

    theta_T = (1.0 - lam) * theta[i] + lam * theta[i + 1]
    psi_T = (1.0 - lam) * psi[i] + lam * psi[i + 1]
    rho_psi_T = (1.0 - lam) * rho[i] * psi[i] + lam * rho[i + 1] * psi[i + 1]
    rho_T = rho_psi_T / psi_T
    return theta_T, psi_T, rho_T

def calibrate_essvi_slice(k_obs, w_obs,
                          prev_theta=None, prev_psi=None, prev_rho=None,
                          rho_grid=None):

    k_obs = np.asarray(k_obs, dtype=float)
    w_obs = np.asarray(w_obs, dtype=float)

    idx_star = int(np.argmin(np.abs(k_obs)))
    k_star = float(k_obs[idx_star])
    theta_star = float(w_obs[idx_star])

    if rho_grid is None:
        rho_grid = np.linspace(-0.99, 0.99, 41)

    big = 1e9
    best_err = np.inf
    best_rho = 0.0
    best_psi = 1e-4
    best_theta = theta_star

    for rho in rho_grid: 
        denom = 1.0 + abs(rho)
        tmp = 4.0 * rho * rho * k_star * k_star / (denom * denom) + 4.0 * theta_star / denom
        if tmp <= 0.0:
            continue
        psi_plus = -2.0 * rho * k_star / denom + np.sqrt(tmp)
        psi_upper = min(psi_plus, 4.0 / denom)
        psi_lower = 1e-8

        if prev_theta is not None and prev_psi is not None and prev_rho is not None:
            psi_lower = max(psi_lower, prev_psi + 1e-8)
            denom1 = 1.0 - rho
            denom2 = 1.0 + rho
            if denom1 <= 0 or denom2 <= 0:
                continue
            psi_min_cal1 = prev_psi * (1.0 - prev_rho) / denom1
            psi_min_cal2 = prev_psi * (1.0 + prev_rho) / denom2
            psi_lower = max(psi_lower, psi_min_cal1, psi_min_cal2)

        if psi_lower >= psi_upper:
            continue

        def obj(psi):
            theta = theta_star - rho * psi * k_star
            if theta <= 1e-8:
                return big + 1e4 * (1e-8 - theta) ** 2

            if prev_theta is not None and theta < prev_theta:
                return big + 1e4 * (prev_theta - theta) ** 2

            w_model = essvi_total_variance(k_obs, theta, psi, rho)
            if not np.all(np.isfinite(w_model)):
                return big
            return np.mean((w_model - w_obs) ** 2)

        res = minimize_scalar(
            obj,
            bounds=(psi_lower, psi_upper),
            method="bounded",
            options={"xatol": 1e-8, "maxiter": 200},)

        if not res.success:
            continue

        psi_opt = float(res.x)
        theta_opt = theta_star - rho * psi_opt * k_star
        if theta_opt <= 0:
            continue

        err = float(res.fun)
        if err < best_err:
            best_err = err
            best_rho = rho
            best_psi = psi_opt
            best_theta = theta_opt

    return best_theta, best_psi, best_rho, k_star, theta_star, best_err


def calibrate_essvi_slices(df):
    Ts = np.sort(df["TTM"].unique())
    params_rows = []

    prev_theta = prev_psi = prev_rho = None

    for i, T in enumerate(Ts):
        sli = df.loc[df["TTM"] == T].copy()
        k_obs = sli["LogMK"].to_numpy()
        w_obs = sli["w"].to_numpy()

        theta, psi, rho, k_star, theta_star, mse = calibrate_essvi_slice(
            k_obs, w_obs,
            prev_theta=prev_theta,
            prev_psi=prev_psi,
            prev_rho=prev_rho,
        )

        rmse = float(np.sqrt(mse))

        params_rows.append({
            "T": T,
            "theta": theta,
            "psi": psi,
            "rho": rho,
            "k_star": k_star,
            "theta_star": theta_star,
            "rmse": rmse,
            "n_quotes": len(sli),
        })

        prev_theta, prev_psi, prev_rho = theta, psi, rho

    essvi_params = (
        pd.DataFrame(params_rows)
        .sort_values("T")
        .reset_index(drop=True)
    )
    return essvi_params

def build_essvi_surface_on_grid(essvi_params,moneyness_grid,ttm_grid,rate):
    Ts = essvi_params["T"].to_numpy()
    theta_arr = essvi_params["theta"].to_numpy()
    psi_arr = essvi_params["psi"].to_numpy()
    rho_arr = essvi_params["rho"].to_numpy()

    moneyness_grid = np.asarray(moneyness_grid, dtype=float)
    log_m = np.log(moneyness_grid)

    rows = []
    for T in ttm_grid:
        theta_T, psi_T, rho_T = interpolate_essvi_params(
            T, Ts, theta_arr, psi_arr, rho_arr
        )

        # Convert spot-moneyness grid (K/S) to log-forward moneyness (K/F)
        # k(T) = log(K/S) - log(F/S) = log(m) - rate*T
        k_grid = log_m - rate * T

        if theta_T <= 0 or psi_T <= 0:
            w_row = np.zeros_like(k_grid, dtype=float)
        else:
            w_row = essvi_total_variance(k_grid, theta_T, psi_T, rho_T)

        for mg, k_val, wv in zip(moneyness_grid, k_grid, w_row):
            rows.append({
                "T": float(T),
                "moneyness": float(mg),   # still store K/S for labeling
                "k": float(k_val),        # now correctly stores log(K/F)
                "w": max(float(wv), 0.0),
            })

    grid_essvi = (
        pd.DataFrame(rows)
        .sort_values(["T", "moneyness"])
        .reset_index(drop=True)
    )
    grid_essvi["iv"] = np.sqrt(
        np.where(grid_essvi["T"] > 0.0, grid_essvi["w"] / grid_essvi["T"], 0.0)
    )
    return grid_essvi

In [10]:

def formating_vol_cube(df,dates):
    vol_cube = np.empty((1, 5, 20, 20))
    
    for date in dates:
        rate = get_rate(df_risk_free_rates, date)
        price = get_price(df,date)
        df_data = df[df["QUOTE_DATE"] == date]
        essvi_params = calibrate_essvi_slices(df_data)
        grid_essvi = build_essvi_surface_on_grid(essvi_params,moneyness_grid=moneyness_grid,ttm_grid=ttm_grid,rate=rate,)
        df_grid_pivot = grid_essvi.pivot(index='T', columns='moneyness', values='iv').sort_index().sort_index(axis=1)
        IV_grid = df_grid_pivot.to_numpy()
        IV_grid_expanded = np.expand_dims(IV_grid, axis=0)
        rate_expanded = np.full(IV_grid_expanded.shape, rate)
        price_expanded = np.full(IV_grid_expanded.shape, price)
        IV_grid_expanded = np.concatenate((TTM_grid_expanded, IV_grid_expanded), axis=0)
        IV_grid_expanded = np.concatenate((IV_grid_expanded, price_expanded), axis=0)
        IV_grid_expanded = np.concatenate((IV_grid_expanded, rate_expanded), axis=0)
        IV_grid_expanded = np.concatenate((IV_grid_expanded, price*M_grid_expanded), axis=0)
        IV_grid_expanded = np.expand_dims(IV_grid_expanded, axis=0)
        vol_cube = np.concatenate((vol_cube, IV_grid_expanded), axis=0)
    vol_cube = np.delete(vol_cube, 0, axis=0)
    return vol_cube

In [11]:
moneyness_grid = np.arange(0.9, 1.09, 0.01)
ttm_grid = np.arange(30, 600 + 1, 30) / 365
M_grid, TTM_grid = np.meshgrid(moneyness_grid, ttm_grid)

In [12]:
M_grid_expanded = np.expand_dims(M_grid, axis=0)
TTM_grid_expanded = np.expand_dims(TTM_grid, axis=0)

In [13]:
df_risk_free_rates = rates_clean(risk_free_rates)

/tmp/ipykernel_47/786814357.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Date"] = pd.to_datetime(df["Date"])


In [14]:
df_risk_free_rates.to_csv("rates_interpolated.csv")

In [15]:
df_option_chains_train = clean_data(option_chains_train)
df_option_chains_rates_train = df_option_chains_train.merge(df_risk_free_rates, left_on="QUOTE_DATE", right_on="Date", how="left")
df_option_chains_rates_clean_train = df_option_chains_rates_cleaning(df_option_chains_rates_train)
dates_train = dates_extraction(df_option_chains_rates_clean_train)
vol_cube_train = formating_vol_cube(df_option_chains_rates_clean_train, dates_train)

In [16]:
df_option_chains_train.to_csv("clean_train_chains.csv")
dates_train.to_csv("dates_train.csv")

In [17]:
df_option_chains_test = clean_data(option_chains_test)
df_option_chains_rates_test = df_option_chains_test.merge(df_risk_free_rates, left_on="QUOTE_DATE", right_on="Date", how="left")
df_option_chains_rates_clean_test = df_option_chains_rates_cleaning(df_option_chains_rates_test)
dates_test = dates_extraction(df_option_chains_rates_clean_test)
vol_cube_test = formating_vol_cube(df_option_chains_rates_clean_test, dates_test)

In [18]:
df_option_chains_test.to_csv("clean_test_chains.csv")
dates_test.to_csv("dates_test.csv")

In [19]:
np.savez_compressed("vol_cube.npz",train=vol_cube_train,test=vol_cube_test)

In [20]:
import numpy as np

def norm_cdf(x: np.ndarray) -> np.ndarray:
    """
    Numerically stable approximation of the standard normal CDF.
    Works on numpy arrays without SciPy.
    """
    x = np.asarray(x, dtype=float)
    sign = np.sign(x)
    x_abs = np.abs(x) / np.sqrt(2.0)

    a1 = 0.254829592
    a2 = -0.284496736
    a3 = 1.421413741
    a4 = -1.453152027
    a5 = 1.061405429
    p  = 0.3275911

    t = 1.0 / (1.0 + p * x_abs)
    erf_approx = 1.0 - (((((a5 * t + a4) * t) + a3) * t + a2) * t + a1) * t * np.exp(-x_abs * x_abs)
    erf_approx *= sign

    return 0.5 * (1.0 + erf_approx)

def black_scholes_call(S, K, T, r, sigma):
    """
    Vectorised Black-Scholes call price.
    S, K, T, r, sigma can be scalars or numpy arrays with broadcastable shapes.
    """
    S = np.asarray(S, dtype=float)
    K = np.asarray(K, dtype=float)
    T = np.asarray(T, dtype=float)
    r = np.asarray(r, dtype=float)
    sigma = np.asarray(sigma, dtype=float)

    eps = 1e-12
    sigma = np.maximum(sigma, eps)
    T = np.maximum(T, eps)
    sqrtT = np.sqrt(T)

    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT

    Nd1 = norm_cdf(d1)
    Nd2 = norm_cdf(d2)

    return S * Nd1 - K * np.exp(-r * T) * Nd2

def check_call_surface_no_arbitrage(
    S0: float,
    r: float,
    T_grid: np.ndarray,
    K_grid: np.ndarray,
    vol_grid: np.ndarray,
    tol: float = 1e-8,) -> dict:

    T_grid = np.asarray(T_grid, dtype=float)
    K_grid = np.asarray(K_grid, dtype=float)
    vol_grid = np.asarray(vol_grid, dtype=float)

    n_T, n_K = vol_grid.shape
    assert T_grid.shape == (n_T,)
    assert K_grid.shape == (n_K,)

    T_mat = T_grid[:, None] * np.ones((1, n_K))
    K_mat = np.ones((n_T, 1)) * K_grid[None, :]

    C = black_scholes_call(S0, K_mat, T_mat, r, vol_grid)

    lower_bound = np.maximum(0.0, S0 - K_mat * np.exp(-r * T_mat))
    upper_bound = S0

    bound_low_viol  = C + tol < lower_bound
    bound_high_viol = C - tol > upper_bound

    dC_dK = np.diff(C, axis=1)
    monoK_viol = dC_dK > tol

    dK = np.diff(K_grid)
    slopes = (C[:, 1:] - C[:, :-1]) / dK[None, :]
    slope_diff = np.diff(slopes, axis=1)
    convex_viol = slope_diff < -tol

    dC_dT = np.diff(C, axis=0)
    calendar_viol = dC_dT < -tol

    return {
        "any_violation": (
            bound_low_viol.any()
            or bound_high_viol.any()
            or monoK_viol.any()
            or convex_viol.any()
            or calendar_viol.any()
        ),
        "bound_low_count": int(bound_low_viol.sum()),
        "bound_high_count": int(bound_high_viol.sum()),
        "monoK_count": int(monoK_viol.sum()),
        "convex_count": int(convex_viol.sum()),
        "calendar_count": int(calendar_viol.sum()),
    }

def check_sample_from_vol_cube(sample, tol: float = 1e-8):
    ch0 = sample[0]   # T grid replicated across strikes
    ch1 = sample[1]   # vol surface
    ch2 = sample[2]   # spot
    ch3 = sample[3]   # r-like scalar (constant on grid)
    ch4 = sample[4]   # K grid replicated across maturities

    T_grid = ch0[:, 0]
    K_grid = ch4[0, :]
    S0 = float(ch2[0, 0])
    r = float(ch3[0, 0])

    return check_call_surface_no_arbitrage(
        S0=S0,
        r=r,
        T_grid=T_grid,
        K_grid=K_grid,
        vol_grid=ch1,
        tol=tol,
    )


In [21]:
def summarize_dataset_no_arb(vol_cube, tol: float = 1e-8):
    n = vol_cube.shape[0]
    summary = {
        "total_samples": n,
        "no_arbitrage_samples": 0,
        "arbitrage_samples": 0,
        "bound_low_total": 0,
        "bound_high_total": 0,
        "monoK_total": 0,
        "convex_total": 0,
        "calendar_total": 0,
    }
    first_arb_idx = None
    first_arb_details = None

    for i in range(n):
        res = check_sample_from_vol_cube(vol_cube[i], tol=tol)
        if res["any_violation"]:
            summary["arbitrage_samples"] += 1
            summary["bound_low_total"]  += res["bound_low_count"]
            summary["bound_high_total"] += res["bound_high_count"]
            summary["monoK_total"]      += res["monoK_count"]
            summary["convex_total"]     += res["convex_count"]
            summary["calendar_total"]   += res["calendar_count"]
            if first_arb_idx is None:
                first_arb_idx = i
                first_arb_details = res
        else:
            summary["no_arbitrage_samples"] += 1

    summary["first_arbitrage_index"] = first_arb_idx
    summary["first_arbitrage_details"] = first_arb_details
    return summary

In [22]:
arbitrage_tests_train = summarize_dataset_no_arb(vol_cube_train)
arbitrage_tests_test = summarize_dataset_no_arb(vol_cube_test)

In [23]:
arbitrage_tests_train

{'total_samples': 1253,
 'no_arbitrage_samples': 1176,
 'arbitrage_samples': 77,
 'bound_low_total': 0,
 'bound_high_total': 0,
 'monoK_total': 0,
 'convex_total': 0,
 'calendar_total': 1811,
 'first_arbitrage_index': 137,
 'first_arbitrage_details': {'any_violation': True,
  'bound_low_count': 0,
  'bound_high_count': 0,
  'monoK_count': 0,
  'convex_count': 0,
  'calendar_count': 45}}

In [24]:
arbitrage_tests_test

{'total_samples': 570,
 'no_arbitrage_samples': 565,
 'arbitrage_samples': 5,
 'bound_low_total': 0,
 'bound_high_total': 0,
 'monoK_total': 0,
 'convex_total': 0,
 'calendar_total': 45,
 'first_arbitrage_index': 128,
 'first_arbitrage_details': {'any_violation': True,
  'bound_low_count': 0,
  'bound_high_count': 0,
  'monoK_count': 0,
  'convex_count': 0,
  'calendar_count': 14}}